크롤링을 연습하기 위해, 네이버 영화 사이트에서 영화의 평점과 평가를 가져오는 것을 연습해 본다.

In [28]:
#필요한 라이브러리 불러오기

from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv

「이스케이프 룸」이라는 영화를 검색해 리뷰로 들어가 보면

https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=180425&target=after&page=1
위와 같은 주소인 것을 볼 수 있다. 여기서 마지막 부분이 page=1로 끝난다는 점을 이용해서
뒤에 붙은 숫자만 바꾸어 주면 평점 목록의 페이지를 넘겨가며 크롤링을 할 수 있을 것 같다

In [37]:
html = urlopen('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=180425&target=after&page=1')
bsObject = BeautifulSoup(html, "html.parser")

html을 봤을때, 각 평가 하나 하나는 <tr> 이라는 태그 안에 들어있음을 볼 수 있다. 
    평점은 그 속에 class=point라는 속성을 가진 <td> 태그 안에 있고
    평가는 class=title 속성을 가진 <td>태그 속에 있는 것을 볼 수 있다.

In [30]:
for x in bsObject.findAll('td',{'class':'title'}):
    print(x.text) #텍스트 부분만 출력


이스케이프 룸
굿 시간때우기 미 ㅕㅅ쥬 
			
			
			
				
				
				
				신고


이스케이프 룸
재미있을 줄 알고 봤더니 그냥 그랬네요...두번은 안본다~~ 
			
			
			
				
				
				
				신고


이스케이프 룸
연기력 1점, 다시 그 소굴로 기어 처들어가는데 웃으면서 감 + 남주도 쉽게 설득당함 
			
			
			
				
				
				
				신고


이스케이프 룸
DVD로 돌아다녀야 할 수준임.진짜 개연성도 없고 스토리도 빈약해서 아무나 2~3일 동안 야 방탈출에다가 살인 접목시켜서 어떻게 줄거리 짜지? 하면 누구나 짤 수 있는 수준임 거기에다가 돈떡칠. 영화 설명 끝 답없음 
			
			
			
				
				
				
				신고


이스케이프 룸
하나도 안무섭네요 재미는 보통 ㅋㅋ 
			
			
			
				
				
				
				신고


이스케이프 룸
시간이 훅지나가는영화 재미납니다 
			
			
			
				
				
				
				신고


이스케이프 룸
쏘우 - 잔인함 = 이스케이프룸 
			
			
			
				
				
				
				신고


이스케이프 룸
와... 굉장히 명작이다...  이 정도는 아니구요...걍 볼만한 수준입니다... 
			
			
			
				
				
				
				신고


이스케이프 룸
흠..잘모르겠네요ㅠㅠㅠ 
			
			
			
				
				
				
				신고


이스케이프 룸
재밌게 봤습니다~~!! 
			
			
			
				
				
				
				신고



위에 항상 붙어나오는 제목과 아래쪽 신고버튼을 정리해 주는 것이 더 깔끔할 것 같다.
수업 시간에 배운 문자열 자르기와 치환을 이용하여 삭제할 수 있을 것 같다.

In [31]:
for x in bsObject.findAll('td',{'class':'title'}):
    text=x.text[:-5] #문자열 자르기를 이용하여 신고 부분을 뺀 부분만 가져온다
    text=text.replace('이스케이프 룸','',1) #평가 안에 영화 제목이 있을 수도 있기 때문에 하나만 바꿔준다.
    print(text.strip())
    

굿 시간때우기 미 ㅕㅅ쥬
재미있을 줄 알고 봤더니 그냥 그랬네요...두번은 안본다~~
연기력 1점, 다시 그 소굴로 기어 처들어가는데 웃으면서 감 + 남주도 쉽게 설득당함
DVD로 돌아다녀야 할 수준임.진짜 개연성도 없고 스토리도 빈약해서 아무나 2~3일 동안 야 방탈출에다가 살인 접목시켜서 어떻게 줄거리 짜지? 하면 누구나 짤 수 있는 수준임 거기에다가 돈떡칠. 영화 설명 끝 답없음
하나도 안무섭네요 재미는 보통 ㅋㅋ
시간이 훅지나가는영화 재미납니다
쏘우 - 잔인함 = 이스케이프룸
와... 굉장히 명작이다...  이 정도는 아니구요...걍 볼만한 수준입니다...
흠..잘모르겠네요ㅠㅠㅠ
재밌게 봤습니다~~!!


In [32]:
for x in bsObject.findAll('td',{'class':'point'}):
    print(x.text) #평점 출력

10
6
3
2
7
10
10
10
2
8


10페이지에 걸쳐 평점과 평가를 리스트로 저장하는 코드를 짜 본다.

In [33]:
text=[]
point=[]  #평가와 평점을 저장할 빈 리스트 생성

link='https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=180425&target=after&page=' #페이지 숫자만 바꿔가며 사용할 수 있다

for i in range(11): #1~10페이지
    html = urlopen('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=180425&target=after&page='+str(i))
    bsObject = BeautifulSoup(html, "html.parser")
    
    for x in bsObject.findAll('td',{'class':'title'}):
        a=x.text[:-5]
        a=a.replace('이스케이프 룸','',1)
        text.append(a.strip())  #append를 이용해 리스트에 원소를 추가할 수 있다.
    
    for x in bsObject.findAll('td',{'class':'point'}):
        point.append(int(x.text)) #숫자를 이용할 수도 있기 때문에 정수형으로 바꿔서 저장

In [38]:
text[:5]

['굿 시간때우기 미 ㅕㅅ쥬',
 '재미있을 줄 알고 봤더니 그냥 그랬네요...두번은 안본다~~',
 '연기력 1점, 다시 그 소굴로 기어 처들어가는데 웃으면서 감 + 남주도 쉽게 설득당함',
 'DVD로 돌아다녀야 할 수준임.진짜 개연성도 없고 스토리도 빈약해서 아무나 2~3일 동안 야 방탈출에다가 살인 접목시켜서 어떻게 줄거리 짜지? 하면 누구나 짤 수 있는 수준임 거기에다가 돈떡칠. 영화 설명 끝 답없음',
 '하나도 안무섭네요 재미는 보통 ㅋㅋ']

In [39]:
point[:5]

[10, 6, 3, 2, 7]

In [36]:
#enumerate를 이용해 1점을 받은 평가만 출력하기

for i,x in enumerate(text):
    if point[i]==1:
        print(x)

생각보다 별로...   영화 첫장면보고 조잡함을 직감했는데... 역시나  ㅠ
재미없음 너무뻔하고 스릴넘치지도않아요
시사회로 봤는데 노잼...
스토리도 없고 개연성도 없고 재미도 없고 의미도 없음.
새벽에 이런영화를 보러간 내가밉다
돈낭비 시간낭비하기에 가성비는 최고인듯
영화관에서 탈출못해서 잠듬-_-;;
진심 재미없고 난잡하기만 하지 긴장감도 없고 단서고머고 지들끼리 떠들다 알아서 단서 기어나옴
ㄹㅇ 정신없음 진심  시간 많으면 보세요
좆노잼이었음 돈아까움
보고왔는데.. 노잼ㅡㅡ
